<a href="https://colab.research.google.com/github/sahusatya055-art/First/blob/main/AISD_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

5. Mini Assignments for Students
New Polysemous Word
Pick another word with multiple meanings (e.g., "bat" or "apple").

Create two sentences where the word has different meanings.
Use BERT to get token embeddings and compute cosine similarity between the two occurrences.
Sentence Similarity with BERT

Take 4 short sentences: two positive, two negative.
Use [CLS] embeddings and build a 4×4 cosine similarity matrix.
Are positive sentences closer to each other than to negative sentences?
GPT Prompt Experiments

Try 3 different prompts with DistilGPT2.
For each, generate 2 samples (change top_k / top_p if you like).
Observe how sensitive GPT is to the prompt wording.
T5 Text Transformation

Use t5-small with prompts like "translate English to German:" or "summarize:".
Compare T5 outputs with GPT outputs on the same text.
Concept Check (Short Written Answer)
In your own words (5–6 lines each):

Explain the difference between encoder-only, decoder-only, and encoder–decoder models.
Why is BERT usually used for understanding tasks and not for long, coherent text generation?



1) New Polysemous Word
Pick another word with multiple meanings (e.g., "bat" or "apple").

Create two sentences where the word has different meanings.
Use BERT to get token embeddings and compute cosine similarity between the two occurrences.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load BERT (DistilBERT)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

# Two sentences with different meanings of "bat"
s1 = "The bat flew out of the cave at night."
s2 = "He hit the ball with a wooden bat."

# Tokenize
enc1 = tokenizer(s1, return_tensors="pt")
enc2 = tokenizer(s2, return_tensors="pt")

with torch.no_grad():
    out1 = model(**enc1)
    out2 = model(**enc2)

# Token embeddings
emb1 = out1.last_hidden_state[0]
emb2 = out2.last_hidden_state[0]

# Tokens
tokens1 = tokenizer.convert_ids_to_tokens(enc1["input_ids"][0])
tokens2 = tokenizer.convert_ids_to_tokens(enc2["input_ids"][0])

# Get index of "bat"
i1 = tokens1.index("bat")
i2 = tokens2.index("bat")

# Get embedding vectors
v1 = emb1[i1]
v2 = emb2[i2]

# Cosine similarity
sim = F.cosine_similarity(v1, v2, dim=0).item()

print("Sentence 1 tokens:", tokens1)
print("Sentence 2 tokens:", tokens2)
print("\nIndex in S1:", i1, "   Index in S2:", i2)
print(f"\nCosine similarity of 'bat' meanings: {sim:.4f}")


Sentence 1 tokens: ['[CLS]', 'the', 'bat', 'flew', 'out', 'of', 'the', 'cave', 'at', 'night', '.', '[SEP]']
Sentence 2 tokens: ['[CLS]', 'he', 'hit', 'the', 'ball', 'with', 'a', 'wooden', 'bat', '.', '[SEP]']

Index in S1: 2    Index in S2: 8

Cosine similarity of 'bat' meanings: 0.6763


2.Sentence Similarity with BERT

Take 4 short sentences: two positive, two negative.
Use [CLS] embeddings and build a 4×4 cosine similarity matrix.
Are positive sentences closer to each other than to negative sentences?

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
from transformers import AutoTokenizer, AutoModel

bert_name = "distilbert-base-uncased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_name)
bert_model = AutoModel.from_pretrained(bert_name).to(device)
bert_model.eval()


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [ ]:
enc = bert_tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to(device)


In [ ]:
# 4 sentences: 2 positive, 2 negative
sentences = [
    "I really enjoyed this movie.",        # positive
    "The food was absolutely wonderful.",  # positive
    "This movie was terrible and boring.", # negative
    "I hated the food, it was awful."      # negative
]

# Tokenize all sentences
enc = bert_tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to(device)

# Run through BERT
with torch.no_grad():
    outputs = bert_model(**enc)

# Extract [CLS] embeddings (position 0)
cls_embeddings = outputs.last_hidden_state[:, 0, :]   # shape: [4, hidden_dim]

# Build 4×4 cosine similarity matrix
sim_matrix = []
for i in range(4):
    row = []
    for j in range(4):
        sim = F.cosine_similarity(cls_embeddings[i], cls_embeddings[j], dim=0).item()
        row.append(round(sim, 4))
    sim_matrix.append(row)

# Print the matrix
print("Cosine Similarity Matrix (4×4):\n")
for row in sim_matrix:
    print(row)

print("\nSentences:")
for i, s in enumerate(sentences):
    print(f"{i+1}. {s}")


Cosine Similarity Matrix (4×4):

[1.0, 0.9727, 0.9719, 0.9615]
[0.9727, 1.0, 0.9598, 0.964]
[0.9719, 0.9598, 1.0, 0.9613]
[0.9615, 0.964, 0.9613, 1.0]

Sentences:
1. I really enjoyed this movie.
2. The food was absolutely wonderful.
3. This movie was terrible and boring.
4. I hated the food, it was awful.


GPT Prompt Experiments

Try 3 different prompts with DistilGPT2.
For each, generate 2 samples (change top_k / top_p if you like).
Observe how sensitive GPT is to the prompt wording.

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM

gpt_name = "distilgpt2"
gpt_tokenizer = AutoTokenizer.from_pretrained(gpt_name)

# Fix GPT-2 pad token
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

gpt_model = AutoModelForCausalLM.from_pretrained(gpt_name).to(device)
gpt_model.eval()


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
prompts = [
    "Artificial intelligence will",
    "In 10 years, students will",
    "The worst thing about social media is"
]

for prompt in prompts:
    print("=" * 80)
    print(f"PROMPT: {prompt}\n")

    for i in range(2):
        inputs = gpt_tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = gpt_model.generate(
                **inputs,
                max_length=30,
                do_sample=True,
                top_k=50 if i == 0 else 10,
                top_p=0.9 if i == 0 else 0.6,
                num_return_sequences=1,
                pad_token_id=gpt_tokenizer.eos_token_id,
            )

        generated = gpt_tokenizer.decode(output_ids[0], skip_special_tokens=True)

        print(f"Sample {i+1}:")
        print(generated)
        print()

print("=" * 80)
print("Observe how prompt wording changes GPT's responses.")


PROMPT: Artificial intelligence will

Sample 1:
Artificial intelligence will make people more human, smarter and more intelligent.

















Sample 2:
Artificial intelligence will be able to create a new kind of artificial intelligence.
















PROMPT: In 10 years, students will

Sample 1:
In 10 years, students will never be able to get into a job where they no longer have access to information, and to be taught that information is

Sample 2:
In 10 years, students will have to take part in the program, which will include a number of classes and classes, including the coursework, the

PROMPT: The worst thing about social media is

Sample 1:
The worst thing about social media is that most people will think it is a bad thing. I don't know if anyone will ever do that, but

Sample 2:
The worst thing about social media is that it is not a social media. It is a social media platform. It is not a social media platform.

Observe how prompt wording changes GPT's responses.


4)T5 Text Transformation

Use t5-small with prompts like "translate English to German:" or "summarize:".
Compare T5 outputs with GPT outputs on the same text.

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

t5_name = "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_name).to(device)
t5_model.eval()


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
text = "Machine learning allows computers to learn from data and improve their performance over time."
print("ORIGINAL TEXT:\n", text)


ORIGINAL TEXT:
 Machine learning allows computers to learn from data and improve their performance over time.


In [ ]:
t5_input_trans = "translate English to German: " + text

enc_trans = t5_tokenizer(t5_input_trans, return_tensors="pt", truncation=True).to(device)

with torch.no_grad():
    trans_ids = t5_model.generate(
        **enc_trans,
        max_length=60,
        num_beams=4,
        early_stopping=True,
    )

t5_german = t5_tokenizer.decode(trans_ids[0], skip_special_tokens=True)
print("\nT5 TRANSLATION (EN → DE):\n", t5_german)



T5 TRANSLATION (EN → DE):
 Automatisches Lernen ermöglicht es Computern, aus Daten zu lernen und ihre Leistung im Laufe der Zeit zu verbessern.


Step 3: GPT – Try to translate the same text

In [ ]:
gpt_prompt_trans = f"Translate this English sentence to German:\n{text}\nGerman:"

inputs = gpt_tokenizer(gpt_prompt_trans, return_tensors="pt").to(device)

with torch.no_grad():
    gpt_ids = gpt_model.generate(
        **inputs,
        max_length=80,
        do_sample=True,
        top_p=0.9,
        top_k=50,
        pad_token_id=gpt_tokenizer.eos_token_id,
    )

gpt_output = gpt_tokenizer.decode(gpt_ids[0], skip_special_tokens=True)
print("\nGPT OUTPUT (Translation-style prompt):\n", gpt_output)



GPT OUTPUT (Translation-style prompt):
 Translate this English sentence to German:
Machine learning allows computers to learn from data and improve their performance over time.
German: machine learning is the key to a successful machine learning project.
Learn more about Machine Learning at www.machinelearning.org.


5) Concept Check (Short Written Answer)
In your own words (5–6 lines each):

Explain the difference between encoder-only, decoder-only, and encoder–decoder models.
Why is BERT usually used for understanding tasks and not for long, coherent text generation?



1. Difference between encoder-only, decoder-only, and encoder–decoder models (5–6 lines)

Encoder-only models (like BERT) read the entire input at once using bidirectional attention, which helps them understand context deeply. They are mainly used for tasks like classification, similarity, and question answering.
Decoder-only models (like GPT) generate text one token at a time and only look at previous tokens (left-to-right). This makes them very good at text generation and prediction.
Encoder–decoder models (like T5 or BART) use an encoder to understand the input and a decoder to produce an output. They are ideal for tasks like translation, summarization, and text transformation.



2. Why BERT is used for understanding, not long text generation (5–6 lines)

BERT is an encoder-only model, so it processes all words together and looks both left and right (bidirectional). Because of this, it cannot generate text step-by-step like GPT does.
BERT is designed to understand meaning, classify text, or produce embeddings—not to continue sentences.
Long, coherent text generation requires an auto-regressive decoder that predicts the next token, which BERT does not have.
This is why GPT-style models are used for generation, while BERT excels in comprehension tasks.